# AWS Athena to analyse Load Balancer access logs
In this notebook, we'll query AWS Athena from Python using the `awswrangler` library. The AWS credentials will be fetched from environment variables.
There has been a traffic increase on the production Network Load Balancer the 5th of october 2024 and we try to understand why by analysing its access logs. 
Details regarding the NLB's access logs can be found here : https://docs.aws.amazon.com/elasticloadbalancing/latest/network/load-balancer-access-logs.html

In [3]:
# Import necessary libraries
import os
import awswrangler as wr
import pandas as pd

# Set up environment variables (optional, in case not already set)
#os.environ['AWS_ACCESS_KEY_ID'] = 'your-access-key-id'
#os.environ['AWS_SECRET_ACCESS_KEY'] = 'your-secret-access-key'
os.environ['AWS_REGION'] = 'eu-west-1'

DATABASE = "access_logs"
TABLE = "access_logs_nlb"
WORKGROUP = "primary"
S3_OUTPUT = os.environ['S3_OUTPUT']
START_TIME_BEFORE_ISSUE = '2024-10-05T01:00:00' # We look also look at data before the issue so to make comparisons. So we look at data from 1 AM to 2 AM, before the issue
END_TIME_BEFORE_ISSUE = '2024-10-05T02:00:00'
START_TIME_DURING_ISSUE = '2024-10-05T02:00:00' # Issue started at 2 AM and ended at 03 AM
END_TIME_DURING_ISSUE = '2024-10-05T03:00:00'
DATE_DAY= '2024/10/05'



### Show the Schema of the data defined in Glue


In [4]:
# show schema of nlb logs
wr.catalog.table(database=DATABASE, table=TABLE)


,Column Name,Type,Partition,Comment
0,type,string,False,
1,version,string,False,
2,time,string,False,
3,elb,string,False,
4,listener_id,string,False,
5,client_ip,string,False,
6,client_port,int,False,
7,target_ip,string,False,
8,target_port,int,False,
9,tcp_connection_time_ms,double,False,


Only date_day is a partition key. We will specify date_day in the WHERE clause of each sql query to reduce the quantity of scanned data in the bucket, so to reduce cost and have faster queries.  

### Execute the query using awswrangler
The Athena workgroup is set to `primary`, and the region is `eu-west-1`. Below we define a function to easily run all our sql queries without the need to repeat the database name and the other parameters each time. 

In [5]:
# Define the function to run Athena query and return Pandas DataFrame
def query_athena(sql_query):
    # Execute the query using awswrangler and return the result as a DataFrame
    df = wr.athena.read_sql_query(
        sql=sql_query,
        database=DATABASE,
        workgroup=WORKGROUP,
        s3_output=S3_OUTPUT,
        ctas_approach=False  
    )
    
    return df

query_athena(f"select 1 ") # to test that the function is working properly


,_col0
0,1


### First view of the data

In [ ]:
start_time = START_TIME_DURING_ISSUE
end_time = END_TIME_DURING_ISSUE
query = f'''
SELECT *
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
'''
query_athena(query)


The query above is a first basic query which shows the content of the access log in pandas format for the time period where the issue appeared. For more informations regarding the columns and their contents, see the table schema above and the explanations of each column here : https://docs.aws.amazon.com/elasticloadbalancing/latest/network/load-balancer-access-logs.html

### Identifying the number of connections to the load balancer before and during the issue 

#### Before the issue

In [ ]:
start_time = START_TIME_BEFORE_ISSUE
end_time = END_TIME_BEFORE_ISSUE
query = f'''
SELECT COUNT(num_lign) as nb_of_requests
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
)
'''
query_athena(query)


#### During the issue

In [ ]:
start_time = START_TIME_DURING_ISSUE
end_time = END_TIME_DURING_ISSUE
query = f'''
SELECT COUNT(num_lign) as nb_of_requests
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
)
'''
query_athena(query)


#### Compare results before and during the issue 

Running the two queries above we indeed confirm that there is 3 times more requests to the load balancer during the issue compared to before the issue

### Identifying how many distinct client ips have done queries to the load balancer before and during the issue 

#### Before the issue

In [ ]:
start_time = START_TIME_BEFORE_ISSUE
end_time = END_TIME_BEFORE_ISSUE
query = f'''
 SELECT COUNT(distinct client_ip ) as nb_of_client_ip
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
)
'''
query_athena(query)


#### During the issue

In [ ]:
start_time = START_TIME_DURING_ISSUE
end_time = END_TIME_DURING_ISSUE
query = f'''
 SELECT COUNT(distinct client_ip ) as nb_of_client_ip
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
)
'''
query_athena(query)


#### Compare results before and during the issue 

There is less client ips making queries during the issue. We suppose this is an abnormal situation, there might be some specific ips generating an unwanted high traffic

### Identifying the number of connections per tcp port

#### Before the issue

In [ ]:
start_time = START_TIME_BEFORE_ISSUE
end_time = END_TIME_BEFORE_ISSUE
query = f'''
SELECT  target_port , COUNT(num_lign) as connections_on_this_port
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
-- LIMIT 10
)
GROUP BY target_port
'''
query_athena(query)


#### During the issue

In [ ]:
start_time = START_TIME_DURING_ISSUE
end_time = END_TIME_DURING_ISSUE
query = f'''
SELECT  target_port , COUNT(num_lign) as connections_on_this_port
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
-- LIMIT 10
)
GROUP BY target_port
'''
query_athena(query)


#### Compare results before and during the issue 

- Port 8080 : same before and during the issue
- Port 443 :  x5 during the issue

As connection increased only for port 443, we now decide to focus our analysis on port 443

### Identifying the number of connections per domain

#### Before the issue

In [ ]:
start_time = START_TIME_BEFORE_ISSUE
end_time = END_TIME_BEFORE_ISSUE
query = f'''
SELECT  domain_name , COUNT(num_lign) as connections_on_this_domain_name
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND target_port = 443
-- LIMIT 10
)
GROUP BY domain_name
'''
query_athena(query)


#### During the issue

In [ ]:
start_time = START_TIME_DURING_ISSUE
end_time = END_TIME_DURING_ISSUE
query = f'''
SELECT  domain_name , COUNT(num_lign) as connections_on_this_domain_name
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND target_port = 443
-- LIMIT 10
)
GROUP BY domain_name
'''
query_athena(query)


#### Compare results before and during the issue 

Most of the new traffic comes for domain name api.XXX.com , so we now focus on this domain name for the analysis below

### Identifying the number of connections per client ip

#### Before the issue

In [ ]:
start_time = START_TIME_BEFORE_ISSUE
end_time = END_TIME_BEFORE_ISSUE
query = f'''
SELECT  client_ip , COUNT(num_lign) as connections_from_this_client_ip
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND target_port = 443
AND domain_name = 'api.XXX.com'
)
GROUP BY client_ip
order by connections_from_this_client_ip desc
limit 30
'''
query_athena(query)


#### During the issue

In [ ]:
start_time = START_TIME_DURING_ISSUE
end_time = END_TIME_DURING_ISSUE
query = f'''
SELECT  client_ip , COUNT(num_lign) as connections_from_this_client_ip
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND target_port = 443
AND domain_name = 'api.XXX.com'

)
GROUP BY client_ip
order by connections_from_this_client_ip desc
limit 30
'''
query_athena(query)


#### Compare results before and during the issue 

Thanks to the queries above we are able to identify the 30 most active client ips during the issue. We will now measure the average connection duration for those 30 most active client ips 

### Identifying the average connection duration for the 30 most active client ips on port 443

#### Before the issue

In [ ]:
start_time = START_TIME_BEFORE_ISSUE
end_time = END_TIME_BEFORE_ISSUE
query = f'''
SELECT client_ip, round(avg(tcp_connection_time_ms)) as avg_tcp_connec_time_ms_port443
FROM access_logs_nlb 
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND target_port = 443
AND domain_name = 'api.XXX.com'
AND client_ip in (
SELECT client_ip FROM (
SELECT  client_ip , COUNT(num_lign) as connections_from_this_client_ip
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND domain_name = 'api.XXX.com'

)
GROUP BY client_ip
order by connections_from_this_client_ip desc
limit 30
))
group by client_ip
order by avg_tcp_connec_time_ms desc
'''
query_athena(query)


#### During the issue

In [ ]:
start_time = START_TIME_DURING_ISSUE
end_time = END_TIME_DURING_ISSUE
query = f'''
SELECT client_ip, round(avg(tcp_connection_time_ms)) as avg_tcp_connec_time_ms_port443
FROM access_logs_nlb 
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND target_port = 443
AND domain_name = 'api.XXX.com'
AND client_ip in (
SELECT client_ip FROM (
SELECT  client_ip , COUNT(num_lign) as connections_from_this_client_ip
FROM (
SELECT row_number()over() as num_lign, time, client_ip, target_ip, target_port, tcp_connection_time_ms, tls_handshake_time_ms, received_bytes, sent_bytes, tls_protocol_version, domain_name, tls_connection_creation_time 
FROM access_logs_nlb
WHERE tls_connection_creation_time > '{start_time}'
AND tls_connection_creation_time < '{end_time}'
AND date_day = '{DATE_DAY}'  -- partition key, mandatory filter to reduce quantity of scanned data and thus reduce costs
AND domain_name = 'api.XXX.com'

)
GROUP BY client_ip
order by connections_from_this_client_ip desc
limit 30
))
group by client_ip
order by avg_tcp_connec_time_ms desc
'''
query_athena(query)


#### Compare results before and during the issue 

We identified four ips which where part of the 30 more actives ips and moreover had a TCP connection duration bigger than 5 minutes, which is very high and abnormal: 
- 145.57.55.2
- 78.58.21.1
- 184.25.32.11
- 47.2.1.2

From our internal database we identified that those ips where owned by the client XXX and start a discussion with this client to understand why so many long queries where coming from those ips. 